In [1]:
from DbConnector import DbConnector
from part2 import Database 

# Set up the program
program = DbConnector()
db = Database()

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('assignment2',)
-----------------------------------------------

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('assignment2',)
-----------------------------------------------



### Task 1

How many users, activities and trackpoints are there in the dataset (after it is inserted into the database)

In [ ]:
# Get the number of Users
users = db.get_table_size("User")
print(f"There are {users[0]} users")

# Get the number of Activities
activities = db.get_table_size("Activity")
print(f"There are {activities[0]} activities")

# Get the number of TrackPoints
trackpoints = db.get_table_size("TrackPoint")
print(f"There are {trackpoints[0]} trackpoints")


### Task 3

Find the top 15 users with the highest number of activities.

In [ ]:
query = "SELECT user_id FROM Activity GROUP BY user_id ORDER BY COUNT(*) DESC LIMIT 15"
program.cursor.execute(query)
rows = program.cursor.fetchall()
text = "Top 15 users with the highest number of activities:\n"

for i, val in enumerate(rows):
    text += f"Top {i+1}. {val[0]}\n"
print(text)

### Task 4

Find all users who have taken a bus.

In [ ]:
# Get the users who have taken the bus
users = db.get_user_taken_bus()
users = "".join([f'\n user: {user[0]}' for user in users[0]])
print(f"The user ids for user that have taken the bus: {users}")

### Task 6

Find activities that are registered multiple times. You should find the query even
if it gives zero result.

In [ ]:
query = "SELECT a.id FROM Activity AS a WHERE EXISTS (SELECT b.id FROM Activity AS b WHERE a.user_id = b.user_id AND a.transportation_mode = b.transportation_mode AND a.start_date_time = b.start_date_time AND a.end_date_time = b.end_date_time AND a.id != b.id)"
program.cursor.execute(query)
rows = program.cursor.fetchall()
print("Find activities that are registered multiple times:\n", rows)

### Task 7

a) Find the number of users that have started an activity in one day and ended the activity the next day.

In [2]:
num_users = db.get_num_user_activity_over_a_day()

print(f"Number of users with activity that ends the next day: {num_users[0]}")

Number of users with activity that ends the next day: 95


b) List the transportation mode, user id and duration for these activities.

In [4]:
users = db.get_user_activity_over_a_day()

users = "".join([f'\n user: {user[0]} transportation mode: {user[1]} duration: {user[2]}' for user in users])
print(f"The user who have an end date one day after the start day: {users}")

[('010', 'train', 995959), ('010', 'train', 871311), ('010', 'train', 69830922), ('010', 'train', 881322), ('010', 'train', 873139), ('010', 'train', 852015), ('010', 'train', 779025), ('010', 'bus', 764824), ('010', 'taxi', 768287), ('010', 'taxi', 765526), ('010', 'train', 767066), ('010', 'subway', 764896), ('010', 'taxi', 766575), ('010', 'taxi', 767060), ('010', 'train', 770115), ('010', 'train', 767011), ('010', 'train', 911606), ('010', 'walk', 765897), ('010', 'train', 70850862), ('010', 'train', 852453), ('010', 'taxi', 769812), ('010', 'train', 767066), ('010', 'taxi', 766713), ('010', 'taxi', 766701), ('010', 'taxi', 765821), ('010', 'taxi', 766101), ('020', 'bike', 861001), ('021', 'car', 820089), ('021', 'walk', 799713), ('052', 'airplane', 781826), ('052', 'bus', 769495), ('052', 'bus', 765895), ('052', 'bus', 764616), ('052', 'walk', 768865), ('052', 'bus', 837790), ('052', 'bus', 765135), ('053', 'walk', 765142), ('056', 'bike', 925959), ('056', 'bike', 925959), ('058',

### Task 9

Find the top 15 users who have gained the most altitude meters.

In [ ]:
query = "SELECT user_id, sum(activity_altitude)*0.304 as altitude_in_meters FROM Activity JOIN (SELECT activity_id, SUM(difference) AS activity_altitude FROM (SELECT activity_id, altitude - LAG(altitude) OVER (PARTITION BY activity_id ORDER BY date_time) AS difference FROM TrackPoint) AS altitude_difference WHERE difference > 0 GROUP BY activity_id) AS difference_table ON Activity.id = difference_table.activity_id GROUP BY user_id ORDER BY altitude_in_meters DESC LIMIT 15"
program.cursor.execute(query)
rows = program.cursor.fetchall()
text = "Find the top 15 users who have gained the most altitude meters\n"
for i in rows:
    text += f"User: {i[0]}, Altitude in meters: {i[1]}\n"
print(text)

### Task 10

Find the users that have traveled the longest total distance in one day for each
transportation mode.

### Task 12

Find all the users who have registered transportation_mode and their most used transportation_mode

Comment on implementation: Although with this dataset, does not need to join User, however in cases where user does not have labels, but still has some activities in Activity with transportation_mode then a join is necessary. For users that have the same number of activities tagged on multiple transportation mode, we've decided to take the first transportation mode in alphabetical order.

In [ ]:
query = "SELECT user_id, transportation_mode FROM ( SELECT user_id, transportation_mode, ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY COUNT(*) DESC, transportation_mode ASC) AS rownum FROM Activity JOIN User ON Activity.user_id = User.id WHERE transportation_mode IS NOT NULL AND User.has_labels=TRUE GROUP BY user_id, transportation_mode) AS activity_grouped WHERE rownum=1 ORDER BY user_id"
program.cursor.execute(query)
rows = program.cursor.fetchall()
text = "Find all the users who have registered transportation_mode and their most used transportation_mode\n"
for row in rows:
    text += f"User {row[0]}, Transportation mode: {row[1]}\n"
print(text)

### End

In [ ]:
# Closing the connection
program.close_connection()